In [15]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].lower()
#     new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower().split()
#     if "television" in new_award_names[i]:
#         new_award_names[i].append("tv")
new_award_names

['best motion picture – drama',
 'best motion picture – musical or comedy',
 'best motion picture – foreign language',
 'best motion picture – animated',
 'best director – motion picture',
 'best actor in a motion picture – drama',
 'best actor in a motion picture – musical or comedy',
 'best actress in a motion picture – drama',
 'best actress in a motion picture – musical or comedy',
 'best supporting actor – motion picture',
 'best supporting actress – motion picture',
 'best screenplay – motion picture',
 'best original score – motion picture',
 'best original song – motion picture',
 'best television series – drama',
 'best television series – musical or comedy',
 'best miniseries or motion picture – television',
 'best actor in a television series – drama',
 'best actor in a television series – musical or comedy',
 'best actor in a miniseries or motion picture – television',
 'best actress in a television series – drama',
 'best actress in a television series – musical or comedy'

In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [3]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[~df.str.contains('think|thinking|should|maybe', case = False)]
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
wins_df = df[df.str.contains('won best|wins best|goes to', case = False)]

wins_df

3490      RT @yeahitsaline_: And the   to "Best Twitter ...
4511      RT @EricaDeLong:  #EricaRedCarpet Dustin Hoffm...
7462      And  goes tooooooooo... Oğlum kara kışta böyle...
9194      Biggest zinger so far at the  goes to Amy Poeh...
9730      Watching the  ! I say silver linings wins best...
                                ...                        
174473    RT @playbill: "Les Miserables" wins Best Motio...
174567    RT @nerdywonka: Ben Affleck won Best Director ...
174576    Best Actor, Drama:   For Film Goes To Daniel D...
174578    Best Actor, Drama:   For Film Goes To Daniel D...
174580    Best Actor, Drama:   For Film Goes To Daniel D...
Name: text, Length: 3806, dtype: object

In [4]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [5]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
#pattern2 = [{"TEXT": "goes to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] might need to split goes to into two "text"
#another pattern: use work of art instead of "Person" to get movies etc too.
#can also try using work of art after "won" to see if that turns up anything else
# POS: very instead of lemma win

In [6]:
matcher = Matcher(nlp.vocab)
matcher.add("Winner", [pattern1])

In [7]:
winners = {}
winners = defaultdict(lambda: 1, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        winners[str(span).strip(punctuation).strip()] = winners[str(span).strip(punctuation).strip()] +1

In [8]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

[('Adele wins Best Original Song', 29), ('Christoph Waltz Wins Best Supporting Actor', 24), ('Christoph Waltz wins Best Supporting Actor', 13), ('Daniel Day-Lewis wins Best Actor Motion Picture - Drama', 13), ('Adele won Best Original Song', 6), ('Christoph Waltz won Best Supporting Actor', 5), ('Ben Affleck wins Best Director - Motion Picture', 5), ('Les Miserables wins Best Motion Picture', 5), ('Adele wins Best Original Song Motion Picture', 4), ('Jennifer Lawrence wins Best Actress Motion Picture Comedy', 4), ('Adele Wins Best Original Song', 4), ('Damian Lewis wins Best Actor TV Series', 3), ('Don Cheadle Wins Best TV Actor - Comedy', 3), ('Mychael Danna wins Best Original Score', 3), ('Ben Affleck wins Best Director prize', 3), ('Christoph Waltz wins Best Supporting Actor - Motion Picture', 2), ('Christoph Waltz WINS Best Supporting Actor', 2), ('Yayyyy Adele wins Best Original Song', 2), ('Christoph Waltz Wins Best Supportin Actor', 2), ('Glad Adele won Best Original Song', 2), 

In [14]:
import Levenshtein

2


In [22]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])

    name_count = [0]*len(new_award_names)
    for i in range(len(new_award_names)):
#         match_length = len(set(new_award_names[i]).intersection(award_phrase))
#         try Levenshtein distance instead, may be more accurate
        
        name_count[i] = Levenshtein.distance(new_award_names[i], award_phrase)

    award_index = np.argmin(name_count)
    # WHAT ABOUT TIE? Neither? Or Both?
    #print(award_index)

#     if name_count[award_index] > 1:       
    if "wins" in split:
        person_index = split.index("wins")
    if "won" in split:
        person_index = split.index("won")
    person = split[:index-1]
    person = " ".join(person)
    if person not in award_list[award_index]:
        award_list[award_index][person] = 1
    else:
        award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

[18, 27, 26, 20, 21, 27, 36, 29, 39, 26, 28, 22, 19, 17, 21, 29, 34, 28, 39, 44, 30, 42, 47, 60, 62]
[19, 28, 27, 20, 17, 26, 37, 28, 39, 17, 19, 19, 23, 23, 22, 31, 32, 29, 40, 43, 32, 42, 45, 55, 57]
[19, 28, 27, 20, 17, 26, 37, 28, 39, 17, 19, 19, 23, 23, 22, 31, 32, 29, 40, 43, 32, 42, 45, 55, 57]
[7, 22, 20, 13, 14, 6, 21, 10, 25, 21, 25, 19, 21, 21, 16, 29, 22, 19, 32, 30, 23, 36, 32, 49, 51]
[18, 27, 26, 20, 21, 27, 36, 29, 39, 26, 28, 22, 19, 17, 21, 29, 34, 28, 39, 44, 30, 42, 47, 60, 62]
[19, 28, 27, 20, 17, 26, 37, 28, 39, 17, 19, 19, 23, 23, 22, 31, 32, 29, 40, 43, 32, 42, 45, 55, 57]
[19, 24, 21, 22, 1, 16, 28, 17, 29, 12, 16, 9, 12, 12, 21, 28, 23, 26, 36, 34, 28, 38, 35, 50, 53]
[8, 20, 19, 11, 11, 19, 31, 21, 33, 19, 21, 13, 17, 16, 19, 29, 27, 28, 38, 38, 31, 41, 40, 57, 59]
[22, 27, 24, 24, 12, 19, 31, 20, 32, 14, 15, 11, 5, 2, 23, 29, 24, 26, 34, 31, 28, 36, 34, 47, 49]
[15, 21, 24, 16, 15, 13, 19, 12, 18, 22, 20, 17, 19, 21, 23, 24, 21, 26, 29, 29, 25, 28, 30, 47, 4

In [20]:
import operator
i = 0
awards = {}
for award in award_list:
    if award != {}:
        awards[i] = max(award.items(), key=operator.itemgetter(1))[0]
    else:
        awards[i] = ''
    i+=1
print(awards)

{0: 'claire danes', 1: '', 2: '', 3: '', 4: 'christoph waltz', 5: 'daniel day-lewis', 6: '', 7: 'jennifer lawrence', 8: '', 9: 'christoph waltz', 10: '', 11: 'quentin tarantino', 12: '', 13: 'adele', 14: 'lena dunham', 15: '', 16: '', 17: '', 18: '', 19: '', 20: '', 21: '', 22: '', 23: '', 24: ''}
